### import dependencies

In [1]:
##### work in this repository was inspired by 
##### https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/blob/main/README_EN.md
##### https://www.youtube.com/watch?v=a99p_fAr6e4


### import dependencies

In [2]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd


# this is just to make sure that the results are reproducible to anyone that runs the code lol.
RANDOM_SEED = 42

#### data formatting
https://devqa.io/python-convert-csv-file-to-list/

In [3]:
import glob
labels = {0: "closed", 1: "open"}

Y_train = [] #1d array, (big_number, 1)
X_train = [] #2d array, (big_number, 42)
def data_append_train(data, X_train = X_train, Y_train = Y_train):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/train"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_train.append([float(num) for num in row])
                Y_train.append(label)
    
    print(X_train[-1])
    print("training set:")
    print(len(X_train))
    print(len(Y_train))    

Y_test = []
X_test = []
def data_append_test(data, X_test = X_test, Y_test = Y_test):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/test"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_test.append([float(num) for num in row])
                Y_test.append(label)
    
    print(X_test[-1])
    print("training set:")
    print(len(X_test))
    print(len(Y_test))    
#i'll only use the train test.
data = "data"
data_append_train(data)
data_append_test(data)




closed
['data/train/closed\\eye.csv', 'data/train/closed\\eye1.csv', 'data/train/closed\\eye2.csv']
3
[270.4326057434082, 254.2082405090332, 11.89279317855835, 316.92705154418945, 253.35582733154297, 7.74164617061615, 284.52354431152344, 257.31865882873535, 4.308481812477112, 301.39739990234375, 256.3886547088623, 3.0293849110603333, 300.3896903991699, 253.87916564941406, 1.3657981157302856, 283.58177185058594, 254.76920127868652, 2.539694309234619, 413.4335708618164, 261.1090564727783, 25.647451877593994, 372.53719329833984, 256.2609672546387, 13.838422298431396, 401.43199920654297, 262.17209815979004, 15.797088146209717, 386.1565399169922, 259.97328758239746, 11.58600926399231, 386.9802474975586, 257.97165870666504, 10.236743688583374, 402.0588684082031, 260.3184986114502, 14.386650323867798]
training set:
1990
1990
open
['data/train/open\\eye.csv', 'data/train/open\\eye1.csv', 'data/train/open\\eye2.csv']
3
[240.9740447998047, 259.1934013366699, 6.445690989494324, 285.66261291503906

### since I already have the training dataset split, I just need to train!

In [4]:
# X_df_train = pd.DataFrame(X_train)
# Y_df_train = pd.DataFrame(Y_train)

# X_df_test = pd.DataFrame(X_test)
# Y_df_test = pd.DataFrame(Y_test)


X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
# 
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)


from sklearn.utils import shuffle

# the random_state is so that both shuffle perform the same.
X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_test,Y_test = shuffle(X_test, Y_test, random_state=0)
# X_train.shape, Y_train.shape
X_train, Y_train


(array([[242.33798981, 207.04376221,  23.31485987, ..., 429.30366516,
         213.49546909, -10.80405235],
        [ 86.97021484, 214.23394203,  42.55360126, ..., 208.25977325,
         186.71218872,  -5.68372548],
        [118.69736671, 216.20329857,  22.80077219, ..., 242.98435211,
         193.09581757,  21.16004467],
        ...,
        [155.37166595, 202.89377689,  21.35821342, ..., 295.25913239,
         171.39677525,  18.77847433],
        [332.76046753,  98.79499912,   8.60808194, ..., 467.31437683,
         101.90653324,  24.38389778],
        [113.01543236,  92.34189749,  37.58370638, ..., 247.63631821,
          75.5294466 ,   6.48359478]]),
 array([0, 1, 0, ..., 0, 1, 1]))

In [5]:
NUM_CLASSES = 2
# model_save_path = "model/"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model/eye_model/model.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    #everytime the accuracy gets better, it saves
    save_freq='epoch',
)


#defining the model 

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras

In [7]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (4475, 36)
Shape of X_test: (800, 36)


#### since I am running binary classification with a sigmoid layer, the input shape is different than sparse categorical crossentropy with softmax

In [8]:
# Our vectorized labels
Y_train = np.asarray(Y_train).astype('float32').reshape((-1,1))
Y_test = np.asarray(Y_test).astype('float32').reshape((-1,1))

In [13]:
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

print(Y_test)

Shape of Y_train: (4475, 1)
Shape of Y_test: (800, 1)
[[0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.

In [10]:

model = Sequential([
    # I will just change the input 
    Dense(units=32, activation='relu', input_shape=(36,)),
    Dense(units=32, activation='relu',
        kernel_regularizer=keras.regularizers.l1_l2(0.05)),
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=16, activation='relu', 
        kernel_regularizer=keras.regularizers.l1_l2(0.01)),
    Dropout(0.2),

    # since I am using sigmoid, the last layer has 1 unit, not two!
    # Dense(units=1, activation = "sigmoid")

    #okay, don't reuse this model
    Dense(units = NUM_CLASSES, activation = "softmax")
])

In [11]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy 
from tensorflow.keras.optimizers import Adam

#even though I should be using binary crossentropy... since two labels. but anyways.
model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [12]:
#fitting
history = model.fit(X_train,Y_train, epochs=150, batch_size=32, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/150
140/140 [==============================] - 11s 65ms/step - loss: 8.6508 - accuracy: 0.5283 - val_loss: 5.9073 - val_accuracy: 0.2975
Epoch 2/150
140/140 [==============================] - 6s 46ms/step - loss: 4.6051 - accuracy: 0.5672 - val_loss: 3.7085 - val_accuracy: 0.4175
Epoch 3/150
140/140 [==============================] - 6s 43ms/step - loss: 3.2451 - accuracy: 0.5718 - val_loss: 2.9933 - val_accuracy: 0.4700
Epoch 4/150
140/140 [==============================] - 6s 42ms/step - loss: 2.6821 - accuracy: 0.5971 - val_loss: 2.5204 - val_accuracy: 0.5638
Epoch 5/150
140/140 [==============================] - 6s 41ms/step - loss: 2.2674 - accuracy: 0.6127 - val_loss: 2.1320 - val_accuracy: 0.5663
Epoch 6/150
140/140 [==============================] - 1s 7ms/step - loss: 1.9430 - accuracy: 0.5996 - val_loss: 1.8519 - val_accuracy: 0.4988
Epoch 7/150
140/140 [==============================] - 6s 45ms/step - loss: 1.6723 - accuracy: 0.6219 - val_loss: 1.6495 - val_accuracy: